In [22]:
import numpy as np
import sklearn
import random
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# Dataset Generations

iF 'RT' value == 'err' in processed data, replace that RT value with PRT value.

In [24]:
import pandas as pd

In [25]:
# read csv file and convert
# into a dataframe object
df = pd.DataFrame(pd.read_csv("LatencyDataaa.csv"))
df_clean = df.dropna().reset_index(drop=True)
pd.set_option('display.max_rows', None)
# show the dataframe
df = df_clean

In [16]:
for i in range(len(df)):
    if df.loc[i, 'RT'] == 'err':
        df.loc[i, 'RT'] = df.PRT.iloc[i]
        
df = df[:-6]
print(df.shape)

(203, 11)


In [6]:
#load the following columns and save them as a new data frame df

df = df.loc[:,['Burst', 'RT',	'MMAP', 'FLAG_45', 'FLAG_20', 'In', 'Out', 'PRT', 'Actual']]

In [7]:
# Do data augmentation
Data=df.to_numpy().astype(float)

# Apply data augmentation
aug_data = []
for i in range(len(Data)):
    for j in range(5): 
        d = Data[i].copy()
        k = [random.randint(-4,4), random.randint(-4,4), random.randint(-1,1), random.randint(-4,4)]
        d[[1,5,6,7]] = Data[i, [1,5,6,7]]+k
        aug_data.append(d[np.newaxis,:])
aug_data = np.concatenate(aug_data)
Data_ = np.concatenate([Data[8:],aug_data])
print(Data_.shape)
Data_

(1210, 9)


array([[256.  ,  63.77,   0.  , ...,  33.5 ,  85.54,  35.  ],
       [256.  ,  63.77,   0.  , ...,  33.2 ,  85.54,  35.  ],
       [256.  ,  63.77,   0.  , ...,  33.4 ,  85.54,  35.  ],
       ...,
       [ 96.  ,  44.33,   1.  , ...,  23.  ,  68.69,  20.  ],
       [ 96.  ,  43.33,   1.  , ...,  24.  ,  69.69,  20.  ],
       [ 96.  ,  47.33,   1.  , ...,  25.  ,  70.69,  20.  ]])

In [8]:
# Class 0 = 0-10
# Class 1 = 10-20
# Class 2 = 20-30
# Class 3 = 30-40
# Class 4 = 40-50
# Class 5 = 50-60

# bins = [0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 85]
# bins = [8, 12, 16, 20, 25, 33, 38, 45, 52, 80]

bins = [0, 5, 10,15,20, 25, 30, 35,40, 45, 50]

Labels=[]
New_data=[]
for i in range(len(Data_)):
    p=Data_[i,-1]

   
    New_data.append(Data_[i])
    
    for idx, aa in enumerate(zip(bins[:-1], bins[1:])):
#         print(aa,idx)
        if aa[0] <= p < aa[1]:
            Labels.append(idx)
        elif p >= bins[-1]:
            Labels.append(len(bins)-1)
            break
        elif p < bins[0]:
            Labels.append(0)
            break
            

Y=np.array(Labels)
X=np.array(New_data)
# X=X[:,:-1]
X= X[0:Y.shape[0], 0:8]
print(X.shape,Y.shape)
print(X.shape)
# print(New_data)
# print(Labels)

(1210, 8) (1210,)
(1210, 8)


In [26]:
my_y = df['Actual']
my_x = df[['Burst', 'RT', 'MMAP', 'FLAG_45', 'FLAG_20', 'In', 'Out', 'PRT']]

In [27]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [28]:
new_data = df[['Burst', 'RT', 'MMAP', 'FLAG_45', 'FLAG_20', 'In', 'Out', 'PRT']]
scaler = StandardScaler()
scaler.fit(new_data)
scaled_data = scaler.transform(new_data)
scaled_data[0]

array([-0.77255843, -0.89033404,  0.81975606,  0.5386311 ,  1.08493471,
       -0.37425698, -0.71036035, -0.70549091])

In [29]:
my_x_array = np.asarray(my_x)
my_x_array[0]

array([96.  , 45.6 ,  1.  ,  1.  ,  1.  , 28.7 , 24.2 , 52.23])

In [14]:
pca = PCA(n_components=8)
pca.fit(scaled_data)
# pca.fit(my_x_array)
x_pca = pca.transform(scaled_data)
x_pca.shape
# y.shape
x_pca[0]

array([ 0.51004364, -1.88712863,  0.38245037,  0.24504862,  0.47520628,
       -0.22194108, -0.52361105, -0.2379442 ])

In [30]:
X_train, X_test, y_train, y_test = train_test_split(my_x, my_y, test_size=0.2, random_state=45,shuffle=True)

In [32]:
clf=[]

# initializing Support Vector Classifier (SVC) from scikit-learn library
CLF=SVC(C=20, kernel='linear',random_state=42)
# CLF=SVC(C=20, kernel='poly', degree=7, random_state=42)

# Training on trainset
CLF.fit(X_train,y_train)

# evaluating on TestSet
score = CLF.score(X_test,y_test)
clf.append([CLF, score])
print(score)

0.8095238095238095


In [65]:
CLF.classes_

array([ 0.,  5., 10., 15., 20., 25., 30., 35., 40., 45., 50.])

# Dict Having Brand Names

In [67]:
brands = ['apple', 'google', 'huawei',  'tecno', 'redmi', 'infinix', 'oneplus',  'oppo', 'realme',  'samsung',  'sony',  'vivo', 'xiaomi', 'zte',  'htc']

mobile_dict = {}
for index, brand in enumerate(brands):
    mobile_dict[index] = brand

print(mobile_dict)


{0: 'apple', 1: 'google', 2: 'huawei', 3: 'tecno', 4: 'redmi', 5: 'infinix', 6: 'oneplus', 7: 'oppo', 8: 'realme', 9: 'samsung', 10: 'sony', 11: 'vivo', 12: 'xiaomi', 13: 'zte', 14: 'htc'}


In [70]:
import pandas as pd

# Read the excel file into a pandas DataFrame
df = pd.read_csv('LatencyDataaa.csv')

# Get the values of the first column
first_column = df.iloc[:,0]

# Get only the first word of each value
first_word = [value.split()[0].lower().strip() for value in first_column]

# Convert to lowercase and remove spaces
processed_names = [name.replace(" ", "").lower() for name in first_word]

# print(processed_names)
df.iloc[:,0] = processed_names
df.iloc[:,0]



0       google
1       google
2       google
3       google
4       google
5       google
6         oppo
7         oppo
8         oppo
9         oppo
10        oppo
11        oppo
12     oneplus
13     oneplus
14     oneplus
15     oneplus
16     oneplus
17     oneplus
18     oneplus
19     oneplus
20     oneplus
21     oneplus
22     oneplus
23     oneplus
24     oneplus
25     oneplus
26     oneplus
27     oneplus
28     oneplus
29     oneplus
30     oneplus
31     oneplus
32     oneplus
33     oneplus
34     oneplus
35     oneplus
36     samsung
37     samsung
38     samsung
39     samsung
40     samsung
41     samsung
42     samsung
43     samsung
44     samsung
45     samsung
46     samsung
47     samsung
48     samsung
49     samsung
50     samsung
51     samsung
52     samsung
53     samsung
54     samsung
55     samsung
56     samsung
57     samsung
58     samsung
59     samsung
60     samsung
61     samsung
62     samsung
63     samsung
64     samsung
65     samsung
66        